# DT Assignment1

# Data Loading

In [2]:
import pandas as pd 
import numpy as np

In [3]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

In [4]:
pd_data.head() # 데이터 확인!

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
def get_gini(df, label):
    ratio = df[label].value_counts(normalize=True)
    gini = 1 - (ratio ** 2).sum()    
    return gini

# gini 계수를 구하는 식을 그대로 구현한 것입니다!

# 1. ratio
# df의 label에서 값의 비율을 확인한 후,
# normalize=TRUE를 통해 개수가 아닌 상대비율을 구합니다

# 2. gini
# gini계수는 1에서 비율의 제곱의 합(시그마)을 뺀 값이므로 그대로 구현했습니다

In [6]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
# for k in c 는 주어진 결과값과 동일한 형식으로 반환하기 위한 코드입니다!
# itertools 활용했습니당 :)

import itertools

def get_binary_split(df, attribute):
    result = []
    
    for i in range(len(df[attribute].value_counts())+1):
        c = list(itertools.combinations(df[attribute].unique(), i))  #조합(combination) 뽑음
        for  k in c:
            result.append(list(k))
        
    return result[1:-1]

In [8]:
get_binary_split(pd_data, "age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [9]:
# labe의 혼용을 막기 위해 target

def get_attribute_gini_index(df, attribute, label):
    dic = {}
    for i in get_binary_split(df, attribute):
        # yes
        new_df1 = df[df[attribute].apply(lambda x: x in i)]
        gini_1 = len(new_df1)/len(df) * get_gini(new_df1, label)
        
        # no
        new_df2 = df[df[attribute].apply(lambda x: x not in i)]
        gini_2 = len(new_df2)/len(df) * get_gini(new_df2, label)
        
        #gini index
        result = gini_1 + gini_2
        dic[tuple(i)] = result
    
    return dic

In [10]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [11]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [12]:
##문제1 답안
get_binary_split(pd_data, "income")

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [13]:
##문제2 답안

my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(pd_data, "income", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(pd_data, "student", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('middle_aged',) : 0.35714285714285715
Min - ('high',) : 0.4428571428571429
Min - ('no',) : 0.3673469387755103
Min - ('fair',) : 0.42857142857142855


age 변수가 가장 중요한 변수임을 알 수 있다.

In [28]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

In [23]:
##문제3 답안

# df 나누기
new_df1 = pd_data[pd_data['age'].apply(lambda x: x in 'middle_aged')]
new_df2 = pd_data[pd_data['age'].apply(lambda x: x not in 'middle_aged')]

In [24]:
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [25]:
# 1. new_df1 : 결과값이 0 

my_dict = get_attribute_gini_index(new_df1, "income", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(new_df1, "student", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(new_df1, "credit_rating", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('high',) : 0.0
Min - ('no',) : 0.0
Min - ('fair',) : 0.0


In [27]:
# 2. new_df2

my_dict = get_attribute_gini_index(new_df2, "income", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(new_df2, "student", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

my_dict = get_attribute_gini_index(new_df2, "credit_rating", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - ('high',) : 0.375
Min - ('no',) : 0.31999999999999984
Min - ('fair',) : 0.4166666666666667


In [29]:
get_attribute_gini_index(pd_data, "income", "class_buys_computer")

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}